In [13]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [14]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas
import numpy
from time import sleep

In [15]:
#the base url for etherscan
baseUrl='https://etherscan.io/'
#the connection pool
pool=urllib3.PoolManager()
def html_convert_top100(tokenid, classname):
    '''
    Get the top 100 owners of the coin
    '''
    global baseUrl
    global pool
    funUrl_chart='token/tokenholderchart/'
    r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
    html=r.data
    soup = BeautifulSoup(html)
    table = soup.find("table", attrs=classname)
    headings = [th.get_text() for th in table.find("tr").find_all("th")]
    for idx in range(len(headings)):
        headings[idx]=str(headings[idx])
    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = (td.get_text() for td in row.find_all("td"))
        ls=list(dataset)
        datasets.append(ls)

    '''
    Clean the data
    '''
    for idx in range(len(datasets)):
        for idxx in range(len(datasets[idx])):
            tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
            tmp=tmp.strip()
            tmp=str(tmp)
            datasets[idx][idxx]=tmp

    '''
    Create pandas dataframe and convert it to float
    '''
    df=pandas.DataFrame(datasets, columns=headings)
    df['Quantity (Token)']=df['Quantity (Token)'].str.replace(',','')
    df['Quantity (Token)']=df['Quantity (Token)'].astype(numpy.float64)
    df['Percentage']=df['Percentage'].str.strip('%')
    df['Percentage']=df['Percentage'].astype(numpy.float64)/100
    return df

def owners_tr(ownerid, tokenname, classname):
    global baseUrl
    global pool
    transUrl='tokentxns?a='
    nextlinks=[]
    nextlinks.append(transUrl+ownerid)
    '''
    trans_dic={
        tx:{
            Value:'', 
            Block:''
        }
    }
    '''
    trans_dic={}
    print tokenname+" Start!"
    while len(nextlinks)>0:
        link=nextlinks.pop()
        r=pool.request('GET',baseUrl+link)
        html=r.data
        soup = BeautifulSoup(html)
        '''
        Get the next link
        '''
        a_tag=soup.find_all('a', id="ContentPlaceHolder1_HyperLinkNext",href=True)
        if a_tag[0]['href']!='#':
            next_link=a_tag[0]['href']
            nextlinks.append(next_link)
        table = soup.find("table", attrs=classname)
        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        for idx in range(len(headings)):
            headings[idx]=str(headings[idx])
            if headings[idx]=='':
                headings[idx]='direction'
        datasets = []
        for row in table.find_all("tr")[1:]:
            dataset = (td.get_text() for td in row.find_all("td"))
            ls=list(dataset)
            datasets.append(ls)
        '''
        Clean the data
        '''
        for idx in range(len(datasets)):
            for idxx in range(len(datasets[idx])):
                tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
                tmp=tmp.strip()
                tmp=str(tmp)
                datasets[idx][idxx]=tmp
        '''
        Create pandas dataframe and convert it to float
        '''
        df=pandas.DataFrame(datasets, columns=headings)
        df['Token']=df['Token'].str.upper()
        df=df.loc[df['Token'] == tokenname]
        df['Value']=df['Value'].str.replace(',','')
        df['Value']=df['Value'].astype(numpy.float64)
        for index, row in df.iterrows():
            if row['Value']!=0:
                if row['direction'] == 'OUT':
                    val=-row['Value']
                else:
                    val=row['Value']
                trans_dic[row['TxHash']]={}
                trans_dic[row['TxHash']]['Value']=val
        
        txurl='tx/'
        for tx in trans_dic:
            block=''
            req=pool.request('GET',baseUrl+txurl+tx)
            html_tx=req.data
            tx_soup=BeautifulSoup(html_tx)
            tx_a_tag=tx_soup.find_all('a',href=True)
            for tag in tx_a_tag:
                if '/block/' in tag['href']:
                    block=str(tag.getText())
                    trans_dic[tx]['Block']=block
        #set a lag here to reduce CPU pressure
        sleep(0.001)
    print tokenname+" Finished!"
    return trans_dic

def ICO_TOKEN(tokenid, tokenname):

    df=html_convert_top100(tokenid, "table table-hover ")
    '''
    Construct the ownership table
    '''
    owners={}
    for index, row in df.iterrows():
        print row['Address']
        if row['Address']=='0xb95313cb20afcbafe9cbb0414edbe436eab0bc88':
            owners[row['Address']]=row['Quantity (Token)']
    print owners
    '''
    Construct the ownership transaction table
    '''
    trans_history={}
    for owner in owners:
        trans_history[owner]=owners_tr(owner,tokenname, 'table table-hover ')
    
    '''
    Backout the transaction history
    '''
    headtable=['Block Height', 'Owner', 'TransactionID', 'TOKEN', 'BALANCE']
    content=[]
    for owner in owners:
        balance=owners[owner]
        trans=trans_history[owner]
        for t in trans:
            entry=[]
            #transaction ID
            TID=t
            #Block Height
            Block=trans[t]['Block']
            balance=balance-trans[t]['Value']
            entry.append(Block)
            entry.append(owner)
            entry.append(TID)
            entry.append(tokenname)
            entry.append(balance)
            content.append(entry)
    dataframe=pandas.DataFrame(content, columns=headtable)
    return dataframe
    
datahistory=ICO_TOKEN('0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0','EOS')
datahistory

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf
0x00000000000000000000000000000000000000b1
0x42da8a05cb7ed9a43572b5ba1b8f82a0a6e263dc
0xf4b51b14b9ee30dc37ec970b50a486f37686e2a8
0x174443351e21d47ed9ab51517a301107d92ede64
0xe2dd88165572932944ba72ab6ed143f9a9855a81
0xb819dfe8927f1055467c957083a752e2f67e4d65
0x0aaca9876ec3fe0245daec6b17c52786800ebb7b
0x6dde928226f1bccf0932396c7f8501c61c04577a
0x8a8f09adb437b2bfdbb4ae53ad82c7791ef443f1
0xd53d9a11dd1fe5ba2dea96cd6c61a7f81dc1c80c
0x9adf5079abf6757acb36216c7c44399090d70f87
0x768a8b7927a3756f659b853d01f370f1d20c82e5
0xe7e561fc67d0008e590fd383fc0fe1571fb7b27e
0x91136d2250fef45d08d3e34bf9e450a7067456d7
0xa27b29df3664fe34a8d942477a8e1279ba0faca7
0x74d909a29c7ed931ac89e52b72948fa63eb48f04
0x2069fcf4b950039a6af59d551b9a3abe81a8a629
0x4efd35e6668339e983134a1d214bdce0f785abb1
0xff0cb0351a356ad16987e5809a8daaaf34f5adbe
0xb3f636cae9e8ad9795d14d3bdda3e382dba47c73
0xa0fb53e1c0c9492b846ee737a3eec6098581c964
0x4752218e54de423f86c0501933917aea08c8fed5
0x9d3319b7f

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is be

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


EOS Finished!


,Block Height,Owner,TransactionID,TOKEN,BALANCE
0,4011493,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x02887c0aeea650e953f662ae516cb2eb31f45c62ee51...,EOS,250973.0
1,4065255,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x273aa3585a1401320470f4442a6f1845a333e4e23f15...,EOS,241973.0
2,4011225,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x596dd35ddf6fe5d25ad5aea2916bd09f212865d6d018...,EOS,233573.0
3,4032547,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0xd2359b9018c6d39c81f88202a0f901ad0d4e5ad06281...,EOS,228573.0
4,4012105,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x871ffd6820d3725a7df21c2a54695382e0f2d93a35ab...,EOS,223573.0
5,4058187,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x6aa2861b9b927b153056d490004fce177c20823dee2e...,EOS,214573.0
6,4012232,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0xce1e19359c8caceeb86592489242a1f22f34eca561d2...,EOS,205573.0
7,4046751,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x0d3b5290eaff4b39ef0f6272ab08e4a816ce5c0c2fc3...,EOS,200573.0
8,4029986,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0xa47a440307e5a3ecfbeb309ea8b2f00d13526617ca32...,EOS,190574.0
9,4011685,0xb95313cb20afcbafe9cbb0414edbe436eab0bc88,0x4ff66f895f9e7248417d23b01ca60577ed67b58f626e...,EOS,181074.0
